In [1]:
import os
import cv2
import numpy as np
import matplotlib.image as mpimg
from scipy.fftpack import dct
import csv
import matplotlib.image as mpimg
from PIL import Image
# from minisom import MiniSom ## Minisom library and module is used for performing Self Organizing Maps
import pandas as pd
from scipy.spatial import distance
np.set_printoptions(suppress=True) #Force-suppress all exponential notation
# import simpsom as sps
import sys
sys.path.append("../")
from mpl_toolkits.mplot3d import Axes3D
from sklearn_som.som import SOM   #***
from sklearn import datasets  
from sklearn.preprocessing import normalize
import seaborn as sns
# from kneed import KneeLocator, DataGenerator as dg 
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
# import tensorflow as tf
from collections import Counter 
import time 
from numba import jit 
import datetime as dt
import seaborn as sns

#### LAB channel ####

In [2]:
path = './All_Images/'
all_images = os.listdir(path)
length = len(all_images)

print(length)
windowsize_r = 64
windowsize_c = 64
dim = (384,512)

names = []

n_blocks = int(dim[0]/windowsize_r * dim[1]/windowsize_c)   
data = np.zeros((n_blocks*length,windowsize_r)) 

counter_n = -1
for i in range(length):
   
    image_path = os.path.join(path, all_images[i])
    img = cv2.imread(image_path)
    
#     try:
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     except:
#         img = img


    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)    
    l_channel,a_channel,b_channel = cv2.split(img)
    
 #   img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)    

    counter_row = -1
    counter_col = -1
    
    for r in range(0,l_channel.shape[0] , windowsize_r):
        counter_row += 1
        counter_col = -1
        for c in range(0,l_channel.shape[1] , windowsize_c):
            
            counter_n += 1
            counter_col += 1
            window = l_channel[r:r+windowsize_r,c:c+windowsize_c]
#             data[counter_n,0] = calcEntropy2dSpeedUp(window) 
            img_dct = abs(dct( dct( window, axis=0, norm='ortho' ), axis=1, norm='ortho' ))

            for j in range(windowsize_r):
                soton =  np.sum(img_dct[0:j+1,j])
                satr = np.sum(img_dct[j,0:j])
                summation = satr + soton
#                 summation=np.log(summation)
                
                data[counter_n,j] = round(summation,2)                
            image_name  = all_images[i].split()  

            names.append(image_name)
            

275


In [3]:
data

array([[8293.92, 2584.17, 1387.97, ...,  536.05,  528.17,  489.9 ],
       [8722.47, 2515.61, 1392.91, ...,  640.55,  551.6 ,  653.83],
       [8317.58,  954.84, 2376.17, ...,  545.83,  558.3 ,  601.08],
       ...,
       [7142.44, 1028.73,  410.39, ...,   17.47,   14.81,   18.51],
       [4569.5 ,  668.06,  286.02, ...,   27.97,   20.3 ,   40.24],
       [1158.23,  872.47,  184.58, ...,   11.73,   10.09,    7.61]])

In [4]:
final_list_matrix = data

In [5]:
final_list_matrix.shape

(13200, 64)

In [6]:
final_list_matrix

array([[8293.92, 2584.17, 1387.97, ...,  536.05,  528.17,  489.9 ],
       [8722.47, 2515.61, 1392.91, ...,  640.55,  551.6 ,  653.83],
       [8317.58,  954.84, 2376.17, ...,  545.83,  558.3 ,  601.08],
       ...,
       [7142.44, 1028.73,  410.39, ...,   17.47,   14.81,   18.51],
       [4569.5 ,  668.06,  286.02, ...,   27.97,   20.3 ,   40.24],
       [1158.23,  872.47,  184.58, ...,   11.73,   10.09,    7.61]])

In [7]:
names = np.array(names)

In [8]:
final_with_name = np.concatenate((names,final_list_matrix),axis=1)

In [9]:
final_with_name

array([['I01.BMP', '8293.92', '2584.17', ..., '536.05', '528.17',
        '489.9'],
       ['I01.BMP', '8722.47', '2515.61', ..., '640.55', '551.6',
        '653.83'],
       ['I01.BMP', '8317.58', '954.84', ..., '545.83', '558.3', '601.08'],
       ...,
       ['i25_08_5.bmp', '7142.44', '1028.73', ..., '17.47', '14.81',
        '18.51'],
       ['i25_08_5.bmp', '4569.5', '668.06', ..., '27.97', '20.3',
        '40.24'],
       ['i25_08_5.bmp', '1158.23', '872.47', ..., '11.73', '10.09',
        '7.61']], dtype='<U32')

In [10]:
final_with_name.shape

(13200, 65)

#### library ####

In [43]:
"""
Script to implement simple self organizing map using PyTorch, with methods
similar to clustering method in sklearn.

@author: Riley Smith
Created: 1-27-21
"""

import numpy as np

class new_som():
    """
    The 2-D, rectangular grid self-organizing map class using Numpy.
    """
    def __init__(self, m=3, n=3, dim=3, lr=0.01, sigma=0.5, max_iter=50000,
                    random_state=42):
        """
        Parameters
        ----------
        m : int, default=3
            The shape along dimension 0 (vertical) of the SOM.
        n : int, default=3
            The shape along dimesnion 1 (horizontal) of the SOM.
        dim : int, default=3
            The dimensionality (number of features) of the input space.
        lr : float, default=1
            The initial step size for updating the SOM weights.
        sigma : float, optional
            Optional parameter for magnitude of change to each weight. Does not
            update over training (as does learning rate). Higher values mean
            more aggressive updates to weights.
        max_iter : int, optional
            Optional parameter to stop training if you reach this many
            interation.
        random_state : int, optional
            Optional integer seed to the random number generator for weight
            initialization. This will be used to create a new instance of Numpy's
            default random number generator (it will not call np.random.seed()).
            Specify an integer for deterministic results.
        """
        # Initialize descriptive features of SOM
        self.m = m
        self.n = n
        self.dim = dim
        self.shape = (m, n)
        self.initial_lr = lr
        self.lr = lr
        self.sigma = sigma
        self.max_iter = max_iter

        # Initialize weights
        self.random_state = random_state
        rng = np.random.default_rng(random_state)
        self.weights = rng.normal(size=(m * n, dim))
        self._locations = self._get_locations(m, n)

        # Set after fitting
        self._inertia = None
        self._n_iter_ = None
        self._trained = False

    def _get_locations(self, m, n):
        """
        Return the indices of an m by n array.
        """
        return np.argwhere(np.ones(shape=(m, n))).astype(np.int64)

    def _find_bmu(self, x):
        """
        Find the index of the best matching unit for the input vector x.
        """
        # Stack x to have one row per weight
        x_stack = np.stack([x]*(self.m*self.n), axis=0)
        ## Calculate distance between x and each weight
        distance = np.linalg.norm(x_stack - self.weights, axis=1)
        # Find index of best matching unit
        return np.argmin(distance)

    def step(self, x):
        """
        Do one step of training on the given input vector.
        """
        # Stack x to have one row per weight
        x_stack = np.stack([x]*(self.m*self.n), axis=0)

        # Get index of best matching unit
        bmu_index = self._find_bmu(x)

        # Find location of best matching unit
        bmu_location = self._locations[bmu_index,:]

        # Find square distance from each weight to the BMU
        stacked_bmu = np.stack([bmu_location]*(self.m*self.n), axis=0)
        bmu_distance = np.sum(np.power(self._locations.astype(np.float64) - stacked_bmu.astype(np.float64), 2), axis=1)

        ## Compute update neighborhood
        neighborhood = np.exp((bmu_distance / (self.sigma ** 2)) * -1)
        local_step = self.lr * neighborhood

        # Stack local step to be proper shape for update
        local_multiplier = np.stack([local_step]*(self.dim), axis=1)

        # Multiply by difference between input and weights
        delta = local_multiplier * (x_stack - self.weights)

        ## Update weights
        self.weights += delta

    def _compute_point_intertia(self, x):
        """
        Compute the inertia of a single point. Inertia defined as squared distance
        from point to closest cluster center (BMU)
        """
        # Find BMU
        bmu_index = self._find_bmu(x)
        bmu = self.weights[bmu_index]
        # Compute sum of squared distance (just euclidean distance) from x to bmu
        return np.sum(np.square(x - bmu))

    def fit(self, X, epochs=150, shuffle=False): ################################## number of epochs
        """
        Take data (a tensor of type float64) as input and fit the SOM to that
        data for the specified number of epochs.

        Parameters
        ----------
        X : ndarray
            Training data. Must have shape (n, self.dim) where n is the number
            of training samples.
        epochs : int, default=1
            The number of times to loop through the training data when fitting.
        shuffle : bool, default True
            Whether or not to randomize the order of train data when fitting.
            Can be seeded with np.random.seed() prior to calling fit.

        Returns
        -------
        None
            Fits the SOM to the given data but does not return anything.
        """
        # Count total number of iterations
        global_iter_counter = 0
        n_samples = X.shape[0]
        total_iterations = np.minimum(epochs * n_samples, self.max_iter)

        for epoch in range(epochs):
            # Break if past max number of iterations
            if global_iter_counter > self.max_iter:
                break

            if shuffle:
                rng = np.random.default_rng(self.random_state)
                indices = rng.permutation(n_samples)
            else:
                indices = np.arange(n_samples)

            # Train
            for idx in indices:
                # Break if past max number of iterations
                if global_iter_counter > self.max_iter:
                    break
                input = X[idx]
                # Do one step of training
                self.step(input)
                ## Update learning rate
                global_iter_counter += 1
                self.lr = (1 - (global_iter_counter / total_iterations)) * self.initial_lr

        # Compute inertia
        inertia = np.sum(np.array([float(self._compute_point_intertia(x)) for x in X]))
        self._inertia_ = inertia

        # Set n_iter_ attribute
        self._n_iter_ = global_iter_counter

        # Set trained flag
        self._trained = True

        return

    def predict(self, X):
        """
        Predict cluster for each element in X.

        Parameters
        ----------
        X : ndarray
            An ndarray of shape (n, self.dim) where n is the number of samples.
            The data to predict clusters for.

        Returns
        -------
        labels : ndarray
            An ndarray of shape (n,). The predicted cluster index for each item
            in X.
        """
        # Check to make sure SOM has been fit
        if not self._trained:
            raise NotImplementedError('SOM object has no predict() method until after calling fit().')

        # Make sure X has proper shape
        assert len(X.shape) == 2, f'X should have two dimensions, not {len(X.shape)}'
        assert X.shape[1] == self.dim, f'This SOM has dimesnion {self.dim}. Received input with dimension {X.shape[1]}'

        labels = np.array([self._find_bmu(x) for x in X])
        return labels

    def transform(self, X):
        """
        Transform the data X into cluster distance space.

        Parameters
        ----------
        X : ndarray
            Data of shape (n, self.dim) where n is the number of samples. The
            data to transform.

        Returns
        -------
        transformed : ndarray
            Transformed data of shape (n, self.n*self.m). The Euclidean distance
            from each item in X to each cluster center.
        """
        # Stack data and cluster centers
        X_stack = np.stack([X]*(self.m*self.n), axis=1)
        cluster_stack = np.stack([self.weights]*X.shape[0], axis=0)

        # Compute difference
        diff = X_stack - cluster_stack

        # Take and return norm
        return np.linalg.norm(diff, axis=2)

    def fit_predict(self, X, **kwargs):
        """
        Convenience method for calling fit(X) followed by predict(X).

        Parameters
        ----------
        X : ndarray
            Data of shape (n, self.dim). The data to fit and then predict.
        **kwargs
            Optional keyword arguments for the .fit() method.

        Returns
        -------
        labels : ndarray
            ndarray of shape (n,). The index of the predicted cluster for each
            item in X (after fitting the SOM to the data in X).
        """
        # Fit to data
        self.fit(X, **kwargs)

        # Return predictions
        return self.predict(X)

    def fit_transform(self, X, **kwargs):
        """
        Convenience method for calling fit(X) followed by transform(X). Unlike
        in sklearn, this is not implemented more efficiently (the efficiency is
        the same as calling fit(X) directly followed by transform(X)).

        Parameters
        ----------
        X : ndarray
            Data of shape (n, self.dim) where n is the number of samples.
        **kwargs
            Optional keyword arguments for the .fit() method.

        Returns
        -------
        transformed : ndarray
            ndarray of shape (n, self.m*self.n). The Euclidean distance
            from each item in X to each cluster center.
        """
        # Fit to data
        self.fit(X, **kwargs)

        # Return points in cluster distance space
        return self.transform(X)

    @property
    def cluster_centers_(self):
        return self.weights.reshape(self.m, self.n, self.dim)

    @property
    def inertia_(self):
        if self._inertia_ is None:
            raise AttributeError('SOM does not have inertia until after calling fit()')
        return self._inertia_

    @property
    def n_iter_(self):
        if self._n_iter_ is None:
            raise AttributeError('SOM does not have n_iter_ attribute until after calling fit()')
        return self._n_iter_


In [12]:
nn = new_som()

In [13]:
change_som = new_som(m=1, n=3, dim=12)

In [14]:
change_som.fit(final_with_name[:,53:65].astype('float'))

In [15]:
change_predictions = change_som.predict(final_with_name[:,53:65].astype('float'))

In [16]:
change_predictions

array([1, 1, 1, ..., 0, 0, 0])

In [17]:
np.where(change_predictions == 0)

(array([   43,   307,   310, ..., 13197, 13198, 13199]),)

In [18]:
clusterr_zero = np.asarray(np.where(change_predictions == 0))  ### tuple to array

In [19]:
clusterr_zero

array([[   43,   307,   310, ..., 13197, 13198, 13199]])

In [20]:
clusterr_zero.size

6461

In [21]:
np.where(change_predictions == 1)

(array([    0,     1,     2, ..., 13044, 13045, 13046]),)

In [22]:
clusterr_one = np.asarray(np.where(change_predictions == 1))  ### tuple to array

In [23]:
clusterr_one

array([[    0,     1,     2, ..., 13044, 13045, 13046]])

In [24]:
clusterr_one.size

4115

In [25]:
np.where(change_predictions == 2)

(array([    3,     4,     5, ..., 13025, 13039, 13040]),)

In [26]:
clusterr_two = np.asarray(np.where(change_predictions == 2))  ### tuple to array

In [27]:
clusterr_two

array([[    3,     4,     5, ..., 13025, 13039, 13040]])

In [28]:
clusterr_two.size

2624

#### Cluster zero ####

In [29]:
clusterr_zero_final = final_with_name[clusterr_zero,:]

In [30]:
clusterr_zero_final

array([[['I01.BMP', '6135.83', '1799.4', ..., '270.81', '317.82',
         '233.58'],
        ['i01_08_1.bmp', '6391.06', '2227.16', ..., '212.27', '172.45',
         '219.04'],
        ['i01_08_1.bmp', '6376.11', '1318.65', ..., '132.05', '179.7',
         '271.26'],
        ...,
        ['i25_08_5.bmp', '7142.44', '1028.73', ..., '17.47', '14.81',
         '18.51'],
        ['i25_08_5.bmp', '4569.5', '668.06', ..., '27.97', '20.3',
         '40.24'],
        ['i25_08_5.bmp', '1158.23', '872.47', ..., '11.73', '10.09',
         '7.61']]], dtype='<U32')

In [31]:
clusterr_zero_final = np.squeeze(clusterr_zero_final, axis=0)

In [32]:
clusterr_zero_final.shape          

(6461, 65)

In [33]:
len(clusterr_zero_final)

6461

#### Cluster one ####

In [34]:
clusterr_one_final = final_with_name[clusterr_one,:]

In [35]:
clusterr_one_final

array([[['I01.BMP', '8293.92', '2584.17', ..., '536.05', '528.17',
         '489.9'],
        ['I01.BMP', '8722.47', '2515.61', ..., '640.55', '551.6',
         '653.83'],
        ['I01.BMP', '8317.58', '954.84', ..., '545.83', '558.3',
         '601.08'],
        ...,
        ['i25_08_2.bmp', '12742.98', '6019.24', ..., '320.43', '190.56',
         '161.81'],
        ['i25_08_2.bmp', '7979.17', '7255.15', ..., '284.7', '254.56',
         '264.75'],
        ['i25_08_2.bmp', '8815.08', '1854.5', ..., '371.12', '578.2',
         '1271.35']]], dtype='<U32')

In [36]:
clusterr_one_final[:,0]

array([['I01.BMP', '8293.92', '2584.17', '1387.97', '1478.87', '1292.96',
        '1027.2', '1071.01', '1090.34', '1264.94', '925.57', '807.05',
        '1034.88', '981.67', '860.14', '979.82', '745.81', '710.52',
        '849.65', '844.0', '976.61', '1005.06', '1058.58', '913.91',
        '994.67', '1049.86', '1160.61', '914.73', '1254.44', '744.78',
        '953.28', '882.99', '701.43', '998.52', '758.93', '812.77',
        '669.76', '713.79', '698.81', '558.4', '675.43', '599.37',
        '603.85', '534.93', '480.05', '466.35', '422.87', '447.53',
        '488.16', '305.64', '460.49', '399.14', '443.3', '470.65',
        '452.28', '465.17', '435.04', '504.79', '478.86', '576.91',
        '515.27', '477.98', '536.05', '528.17', '489.9']], dtype='<U32')

In [37]:
clusterr_one_final = np.squeeze(clusterr_one_final, axis=0)

In [38]:
clusterr_one_final.shape          

(4115, 65)

#### Cluster two ####

In [39]:
clusterr_two_final = final_with_name[clusterr_two,:]

In [40]:
clusterr_two_final

array([[['I01.BMP', '8093.84', '1961.58', ..., '916.73', '573.47',
         '618.16'],
        ['I01.BMP', '6949.53', '2011.85', ..., '870.75', '699.0',
         '865.59'],
        ['I01.BMP', '9216.8', '454.58', ..., '725.31', '638.3',
         '685.34'],
        ...,
        ['i25_08_2.bmp', '7207.86', '774.48', ..., '690.71', '553.49',
         '598.82'],
        ['i25_08_2.bmp', '7132.67', '6275.03', ..., '599.46', '494.71',
         '529.74'],
        ['i25_08_2.bmp', '7983.34', '1417.09', ..., '460.75', '601.67',
         '1296.99']]], dtype='<U32')

In [41]:
clusterr_two_final = np.squeeze(clusterr_two_final, axis=0)

In [42]:
clusterr_two_final.shape          

(2624, 65)